# Tangle surface

A difficult example is the so-called "tangle surface" (see, e.g., [here](http://paulbourke.net/geometry/tangle/)), given by

$$(x^4 + y^4 + z^4  - 5(x^2 + y^2 + z^2) + 11.8 = 0.$$

This turns out to be quite difficult to calculate with interval methods, as reflected by the increased calculation times.

In [15]:
using ValidatedNumerics, IntervalConstraintProgramming

In [4]:
tangle = @constraint (x^4 + y^4) + z^4  - 5*((x^2 + y^2) + z^2) + 11.8 == 0.0

X = IntervalBox(-5..5, -5..5, -5..5)

@time paving = pave(tangle, X, 1);


  9.590588 seconds (17.55 M allocations: 663.344 MB, 3.43% gc time)


In [12]:
@time refine!(paving, 0.25)

inner = paving.inner
boundary = paving.boundary;

227.004031 seconds (453.81 M allocations: 16.755 GB, 3.37% gc time)


In [24]:
length(paving.boundary)

9240

In [5]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

In [21]:
positions = Point{3, Float32}[(mid(x)) for x in vcat(inner, boundary)]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in vcat(inner, boundary)]

zs = Float32[x[3] for x in positions]
minz = minimum(zs)
maxz = maximum(zs)

xs = Float32[x[1] for x in positions]
minx = minimum(xs)
maxx = maximum(xs)

colors = RGBA{Float32}[RGBA( (zs[i]-minz)/(maxz-minz), (xs[i]-minx)/(maxx-minx), 0f0, 0.3f0)
        for i in 1:length(inner)+length(boundary)]
#colors2 = RGBA{Float32}[RGBA( 0.5f0, 0.5f0, 0.5f0, 0.02f0) for x in boundary];
#colors = vcat(colors1, colors2);

In [22]:
cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

_view(vis)


@async renderloop(window)

Task (runnable) @0x000000011c886f50